## Clase 17
### Aplicaciones

La Par Swap Rate (SPR) se obtiene igualando el valor presente (PV) de la pata fija y de la pata flotante, descomponiendo cada una:

---

**1. Pata fija**
Se trata de una serie de cupones periódicos de magnitud $N \times S$, donde

* $N$ = notional (por ejemplo 1 000 000 USD)
* $S$ = tasa fija anual que buscamos
* $\delta_i = T_i - T_{i-1}$ = longitud del período $i$ en años (p.ej. 0.5 para semestral)
* $P(0,T_i)$ = discount factor para el tiempo $T_i$.

El valor presente de todos esos cupones es

$$
\mathrm{PV}_\text{fixed}
\;=\;
N \; \sum_{i=1}^n \bigl(S \,\delta_i\bigr)\;P(0,T_i)
\;=\;
N\,S\;\sum_{i=1}^n \delta_i\,P(0,T_i).
$$

> **Comentario**: no incluimos intercambio de principal al final porque en un plain‐vanilla swap solo se intercambian intereses.

---

**2. Pata flotante**
Bajo la misma curva de descuento (y asumiendo que el índice se reinvierte a esa curva), la pata flotante tiene por construcción valor inicial cero si la tasa flotante se “resetea” exactamente a la par. Entonces

$$
\mathrm{PV}_\text{float}
\;=\;
N \bigl(1 - P(0,T_n)\bigr).
$$

La explicación rápida es que al inicio recibes $N$ inmediato (intercambio de principal teórico) y al vencimiento pagas $N \times P(0,T_n)$ descontado a hoy, con lo que el neto es $N - N\,P(0,T_n)$.

---





**3. Ecuación para la tasa par**
Para que el valor neto inicial del swap sea cero (swap “at‐the‐market”), igualamos

$$
\mathrm{PV}_\text{fixed}
\;=\;
\mathrm{PV}_\text{float}
\qquad\Longrightarrow\qquad
N\,S\sum_{i=1}^n \delta_i\,P(0,T_i)
\;=\;
N\bigl(1 - P(0,T_n)\bigr).
$$

De aquí despejamos

$$
\boxed{
S
\;=\;
\frac{1 - P(0,T_n)}{\displaystyle \sum_{i=1}^n \delta_i\,P(0,T_i)}
}.
$$

---



**4. Ejemplo numérico con curva flat al 3.50 %**

* Plazo: 5 años, semestral $\Rightarrow$ $\delta_i=0.5$, $n=10$.
* Discount factor: $P(0,t)=e^{-0.035\,t}$.

| $T_i$ (años) | $P(0,T_i)$                   | $\delta_i\,P(0,T_i)$     |
| -----------: | ---------------------------- | ------------------------ |
|          0.5 | $e^{-0.035\cdot 0.5}=0.9828$ | $0.5\times0.9828=0.4914$ |
|          1.0 | 0.9657                       | 0.4828                   |
|            … | …                            | …                        |
|          5.0 | 0.8395                       | 0.4198                   |

Sumando todos los $\delta_i P(0,T_i)$ obtenemos

$$
\sum_{i=1}^{10} \delta_i\,P(0,T_i)\approx 4.5469.
$$

Y $P(0,5)=e^{-0.035\cdot5}\approx0.8395$.

* **PV pata flotante**

$$
\mathrm{PV}_\text{float}
= N\bigl(1 - 0.8395\bigr)
=1\,000\,000\times0.1605\approx160\,543\;\text{USD}.
$$

* **PV pata fija** (en función de una tasa $S$):

$$
\mathrm{PV}_\text{fixed}
=1\,000\,000\times S \times 4.5469.
$$

Igualando a 160 543 USD, despejamos

$$
S
=\frac{160\,543}{1\,000\,000\times4.5469}
\approx0.035303
\quad\widehat{=}\;3.5303\%\!,
$$

que coincide con la fórmula teórica
$\;S=(1 - 0.8395)/4.5469$.



---

**5. Resumen y Aplicación en Python**

* El **PV fijo** es la suma descontada de los cupones: $\;N\,S\sum \delta_iP(0,T_i)$.
* El **PV flotante** es $N(1 - P(0,T_n))$.
* Igualando ambos se obtiene la **tasa par**

  $$
    S
    =\frac{1 - P(0,T_n)}{\sum_{i=1}^n \delta_i\,P(0,T_i)}.
  $$

In [5]:
import math
from typing import List

def discount_factor_flat(rate: float, time: float) -> float:
    """
    Curva plana con capitalización continua:
        P(0, time) = exp(-rate * time)

    :param rate: tipo de interés anual (por ejemplo, 0.035 para 3.5%)
    :param time: tiempo en años
    :return: factor de descuento P(0,time)
    """
    return math.exp(-rate * time)

def generate_schedule(tenor_years: float, payments_per_year: int) -> List[float]:
    """
    Genera los tiempos de pago desde t=0 hasta t=tenor_years,
    en intervalos regulares de tamaño 1/payments_per_year.

    :param tenor_years: plazo total en años (p.ej., 5.0)
    :param payments_per_year: número de pagos por año (p.ej., 2 para semestral)
    :return: lista de tiempos [0.0, 0.5, 1.0, ..., tenor_years]
    """
    n_periods = int(tenor_years * payments_per_year)
    delta = 1.0 / payments_per_year
    return [i * delta for i in range(n_periods + 1)]

def pv_fixed_leg(
    notional: float,
    fixed_rate: float,
    times: List[float],
    discount_rate: float
) -> float:
    """
    Calcula el valor presente de la pata fija de un swap:
        PV_fixed = N * S * Σ_{i=1}^n [ δ_i * P(0,T_i) ]

    :param notional: monto nocional (N)
    :param fixed_rate: tasa fija anual (S)
    :param times: lista de tiempos de pago (incluye t=0 y t=T_n)
    :param discount_rate: tipo de la curva plana para P(0,t)
    :return: valor presente de la pata fija
    """
    pv_coupons = 0.0
    for i in range(1, len(times)):
        delta_i = times[i] - times[i-1]
        P_t = discount_factor_flat(discount_rate, times[i])
        pv_coupons += delta_i * P_t

    return notional * fixed_rate * pv_coupons

def pv_floating_leg(
    notional: float,
    times: List[float],
    discount_rate: float
) -> float:
    """
    Calcula el valor presente de la pata flotante de un swap:
        PV_float = N * (1 - P(0, T_n))

    :param notional: monto nocional (N)
    :param times: lista de tiempos de pago (incluye t=0 y t=T_n)
    :param discount_rate: tipo de la curva plana para P(0,t)
    :return: valor presente de la pata flotante
    """
    Tn = times[-1]
    P_Tn = discount_factor_flat(discount_rate, Tn)
    return notional * (1.0 - P_Tn)

def par_swap_rate(
    times: List[float],
    discount_rate: float
) -> float:
    """
    Calcula la tasa par S que iguala PV_fixed = PV_float:
        S = (1 - P(0,T_n)) / Σ_{i=1}^n [ δ_i * P(0,T_i) ]

    :param times: lista de tiempos de pago (incluye t=0 y t=T_n)
    :param discount_rate: tipo de la curva plana para P(0,t)
    :return: tasa par S
    """
    Tn = times[-1]
    P_Tn = discount_factor_flat(discount_rate, Tn)
    numerator = 1.0 - P_Tn

    denominator = 0.0
    for i in range(1, len(times)):
        delta_i = times[i] - times[i-1]
        P_t = discount_factor_flat(discount_rate, times[i])
        denominator += delta_i * P_t

    return numerator / denominator



In [6]:

# Parámetros del swap
notional = 1_000_000       # USD
tenor = 5.0                # años
payments_per_year = 2      # semestral
flat_rate = 0.035          # 3.50% anual continuo

# 1) Generar el calendario de pagos
times = generate_schedule(tenor_years=tenor,
                            payments_per_year=payments_per_year)
times


[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [7]:

# 2) Calcular la tasa par que iguala ambas patas
swap_rate_par = par_swap_rate(times, discount_rate=flat_rate)
print(f"Tasa par (5Y semestral, curva flat 3.50%): {swap_rate_par*100:.4f}%")


Tasa par (5Y semestral, curva flat 3.50%): 3.5308%


In [8]:

# 3) Calcular PV de la pata fija usando la tasa par
pv_fixed = pv_fixed_leg(notional=notional,
                        fixed_rate=swap_rate_par,
                        times=times,
                        discount_rate=flat_rate)


In [9]:

# 4) Calcular PV de la pata flotante
pv_float = pv_floating_leg(notional=notional,
                            times=times,
                            discount_rate=flat_rate)

# 5) Mostrar resultados y verificar que PV_fixed ≈ PV_float
print(f"PV pata fija  : {pv_fixed:,.2f} USD")
print(f"PV pata flot. : {pv_float:,.2f} USD")
print(f"Diferencia    : {pv_fixed - pv_float:,.2e} USD  (≈ 0)\n")


PV pata fija  : 160,542.98 USD
PV pata flot. : 160,542.98 USD
Diferencia    : 0.00e+00 USD  (≈ 0)



In [10]:

# 6) Valorar swap con una tasa fija distinta (por ejemplo, 4.00%)
custom_fixed_rate = 0.04  # 4.00%
pv_fixed_custom = pv_fixed_leg(notional, custom_fixed_rate, times, flat_rate)
pv_swap_custom = pv_fixed_custom - pv_float
print(f"Si la fija es {custom_fixed_rate*100:.2f}% → PV swap = {pv_swap_custom:,.2f} USD")

Si la fija es 4.00% → PV swap = 21,333.96 USD
